In [1]:
#Importing the required python libraries
import numpy as np
import pandas as pd
import math

In [2]:
#Taking input from the csv in preferred format
trainingData = pd.read_csv("D:\ML_data_letter\A_Z Handwritten Data\A_Z Handwritten Data.csv", header=None)
trainingData = trainingData.to_numpy()

In [3]:
#Scaling down the color scale values to a bin size of 16
l = [1];
for i in range(784):
    l.append(16/256)
l = np.array(l)
trainingData = trainingData.astype('float64')
trainingData*= l
trainingData = trainingData.astype('int32')

In [4]:
#Shuffling the training data set and splitting in the ratio of 1:9 as test and training
x = trainingData
np.random.shuffle(x)
train,test = np.split(x,[int(0.9 * len(x))])
#train = trainingData  
A= train[train[:,0].argsort()]

In [5]:
#Using a dictionary data type to store the key value pairs as labels and their frequency
dic={}

for i in A[:,0]:
    if i in dic:
        dic[i]+=1
    else:
        dic[i] = 1
sum = 0
for i in dic:
    sum+=dic[i]

#Defining an array containing the prior probabilities
prob_arr = []
for i in range(26):
    prob_arr.append(dic[i]/sum)

In [6]:
#Defining a 3d array to calculate conditional probabilities of the pixels
arrayOfPixelProbGivenLabels = []
a=0                #For determining the intial iteration from where the label starts
b=dic[0]           #For determining the final iteration where the label ends
for i in range(26):#iterating through all the labels
    impArra=[]     #For storing the probabilities of 16 bins of 784 pixels for the ith label
    for j in range(1,785):#iterating through all the pixels
        Arra = []         #For storing the probability of the 16 bins for jth pixel of ith label
        dt ={}            #For keeping the count of each bin for the jth pixel of ith label
        for k in range(16):#iterating through all the bins
                dt[k] = 0
        for k in range(a,b):
            dt[A[k][j]] += 1   #counting the occurence of the kth bin for jth pixel of the ith label
            
        for k in dt:
            Arra.append((dt[k]+1)/(dic[i]+16))  #Smoothing
        impArra.append(Arra)
    arrayOfPixelProbGivenLabels.append(impArra)
    if i!=25:
        a=b
        b+=dic[i+1]
tes = np.array(arrayOfPixelProbGivenLabels)

In [7]:
#Storing the prior probabilities in a csv file 
file = open('trained_model1.csv', 'w')
for i in range(len(prob_arr)):
    prob_arr[i] = str(prob_arr[i])
line = ','.join(prob_arr)
file.write(line)
file.close()

In [8]:
#Storing the conditional probabilities of each bin in each pixel of each label in a csv file
file = open('trained_model2.csv', 'w')
for i in arrayOfPixelProbGivenLabels:
    j = []
    for k in range(len(i)):
        for g in range(16):
            j.append(str(i[k][g]))
    line = ','.join(j)
    file.write(line)
    file.write('\n')
file.close()

In [9]:
#Writing the shuffled test data into a csv file to use later
file = open('test_model.csv', 'w')
for i in test:
    j = []
    for k in range(len(i)):
        j.append(str(i[k]))
    line = ','.join(j)
    file.write(line)
    file.write('\n')
file.close()

In [10]:
#Getting back all the data back from all the above just created csv files.
trained_model = pd.read_csv("trained_model2.csv", header=None)
test_data = pd.read_csv("test_model.csv", header=None)
prob_arr = pd.read_csv("trained_model1.csv",header=None)

In [11]:
#Converting everything to np.array
trained_model = trained_model.to_numpy()
test_data = test_data.to_numpy()
test_data = test_data
trained_model = trained_model.reshape(26,784,16)
prob_arr = prob_arr.to_numpy()

In [12]:
prob_arr=np.log(prob_arr) #Taking logarithms of probabilities to handel exceptionally small numbers
count1=0 #for counting the no. of labels
count2=0 #for counting the no. of correct predictions
trained_model = np.log(trained_model) #Taking logarithms of probabilities to handel exceptionally small numbers
for i in test_data:
    count1+=1
    label = i[0] #The actual label
    numar=[] #for storing the final estimated probabilities of each label
    for j in range(26):
        st = trained_model[j]
        #sn is the sum of logs
        sn = 0
        for k in range(1,785):
            sn+=st[(k-1)][i[k]]
        
        sn+=prob_arr[0][j]
        numar.append(sn)
    maxnum = numar[0] # for storing maximum probability inside numar
    index = 0; # for storing the label with maximum probability
    for i in range(len(numar)):
        if(numar[i]>maxnum):
            maxnum = numar[i]
            index = i
    if index == label: #For counting accuracy
        count2+=1
print("Accuracy is: ",(count2/count1)*100)

Accuracy is:  72.10975675240294
